In [468]:
import pandas as pd
import numpy as np

In [469]:
X_train = pd.read_csv('TinyMNIST/trainData.csv', header=None)
X_test = pd.read_csv('TinyMNIST/testData.csv', header=None)
y_train = pd.read_csv('TinyMNIST/trainLabels.csv', header=None)
y_test = pd.read_csv('TinyMNIST/testLabels.csv', header=None)

In [470]:
def normalize(df):
    new_df = df.copy()
    for col in df.columns:
        new_df[col] -= df[col].mean()
        new_df[col] /= df[col].std()
    return new_df.dropna(axis=1)

In [471]:
X_train['Class'] = y_train

In [472]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,187,188,189,190,191,192,193,194,195,Class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.118627,0.067647,0.0,0.0,0.0,0.0,0.0,7.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,6.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,5.0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,8.0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,6.0
4998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,9.0


In [476]:
X_train[X_train['Class']==7]

KeyError: 'Class'

In [473]:
X_train = normalize(X_train)

In [474]:
dfs = []
for i in range(10):
    to_w = X_train[X_train['Class'] == i]
    idxs = to_w.index
    clas = to_w['Class']
    to_w = to_w.drop(columns=['Class'])
#     to_w = normalize(to_w)
#     sigma = np.cov(to_w)
#     evals, evecs = np.linalg.eigh(sigma)
#     to_w = np.diag(np.power(evals, -1/2)) @ evecs.T @ to_w

    to_w = to_w.to_numpy()
    df = pd.DataFrame(to_w, index=idxs)
#     df['Class'] = clas
    dfs.append(df)
    
df = pd.concat(dfs)
X_train = df.sort_index()

In [475]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,166,167,168,169,170,171,172,173,174,175


In [465]:
class NBC:
    def fit(self, X, y):
        X_matrix = X.to_numpy()
        
#         X_matrix_whitened = 
        
        num_samples, num_features = X.shape
        self.classes = np.unique(y)
        self.priors = y[0].value_counts() / num_samples
        self.means = X.groupby(y[0]).mean().to_numpy()
        self.vars = X.groupby(y[0]).std().to_numpy()
    
    
    def predict(self, X_test):
        return [self._predict(x) for x in X_test]
        
    def _predict(self, x):
        posteriors = []
        for c in self.classes:
            c = int(c)
            s_c = self.vars[c]
            mean_c = self.means[c]
            
            non_zero_vars = s_c != 0
            s_c = s_c[non_zero_vars]
            mean_c = mean_c[non_zero_vars]
            x_c = x[non_zero_vars]
            
            likelihoods = np.exp(-(x_c-s_c)**2 / (2 * s_c)) 
            posterior = np.sum(np.log(likelihoods)) + np.log(self.priors[c])
            posteriors.append(posterior)
            
        return self.classes[np.argmax(posteriors)]

nbc = NBC()
nbc.fit(X_train, y_train)

In [466]:
predicts = nbc.predict(X_test.to_numpy())
accuracy = np.sum(predicts == y_test.to_numpy().reshape(1,-1))
accuracy / len(predicts)

IndexError: index 0 is out of bounds for axis 0 with size 0

# Sklearn

In [406]:
X_train = pd.read_csv('TinyMNIST/trainData.csv', header=None)
X_test = pd.read_csv('TinyMNIST/testData.csv', header=None)
y_train = pd.read_csv('TinyMNIST/trainLabels.csv', header=None)
y_test = pd.read_csv('TinyMNIST/testLabels.csv', header=None)
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [407]:
predicts = gnb.fit(X_train.to_numpy(), y_train.to_numpy().reshape(-1)).predict(X_test.to_numpy())
accuracy = np.sum(predicts == y_test.to_numpy().reshape(1,-1))
accuracy / len(predicts)

0.5068